## Summarising, Aggregating, and Grouping data in Python Pandas

![image](https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2016/03/pandas_aggregation-1140x456.png)

### Data

The dataset contains 830 entries from my mobile phone log spanning a total time of 5 months. 
The main columns in the file are:

- **date**: The date and time of the entry
- **duration**: The duration (in seconds) for each call, the amount of data (in MB) for each data entry, and the number of texts sent (usually 1) for each sms entry.
- **item**: A description of the event occurring – can be one of call, sms, or data.
- **month**: The billing month that each entry belongs to – of form ‘YYYY-MM’.
- **network**: The mobile network that was called/texted for each entry.
- **network_type**: Whether the number being called was a mobile, international (‘world’), voicemail, landline, or other (‘special’) number.

The CSV file can be loaded into a pandas DataFrame using the `pandas.DataFrame.from_csv()` function, and looks like this:

In [1]:
import pandas as pd

# Load data from csv file
data = pd.read_csv('https://shanelynnwebsite-mid9n9g1q9y8tt.netdna-ssl.com/wp-content/uploads/2015/06/phone_data.csv')

data.head()

index            date  duration  item    month   network network_type
0      0  15/10/14 06:58    34.429  data  2014-11      data         data
1      1  15/10/14 06:58    13.000  call  2014-11  Vodafone       mobile
2      2  15/10/14 14:46    23.000  call  2014-11    Meteor       mobile
3      3  15/10/14 14:48     4.000  call  2014-11     Tesco       mobile
4      4  15/10/14 17:27     4.000  call  2014-11     Tesco       mobile

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 7 columns):
index           830 non-null int64
date            830 non-null object
duration        830 non-null float64
item            830 non-null object
month           830 non-null object
network         830 non-null object
network_type    830 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 45.5+ KB


### Manipulating Dates

We see **date** as an object. The **date** column can be parsed using the extremely handy `dateutil` library.

In [3]:
import dateutil

# Convert date from string to date times
data['date'] = data['date'].apply(dateutil.parser.parse, dayfirst=True)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data columns (total 7 columns):
index           830 non-null int64
date            830 non-null datetime64[ns]
duration        830 non-null float64
item            830 non-null object
month           830 non-null object
network         830 non-null object
network_type    830 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 45.5+ KB


See the difference!

### Summarising the DataFrame

In [5]:
# How many rows the dataset
data['item'].count()

830

In [6]:
# What was the longest phone call / data entry?
data['duration'].max()

10528.0

In [7]:
# How many seconds of phone calls are recorded in total?
data['duration'][data['item'] == 'call'].sum()

92321.0

In [8]:
# How many entries are there for each month?
data['month'].value_counts()

2014-11    230
2015-01    205
2014-12    157
2015-02    137
2015-03    101
Name: month, dtype: int64

In [9]:
# Number of non-null unique network entries
data['network'].nunique()

9

### Summarising Groups in the DataFrame

In [10]:
data.groupby(['month']).groups.keys()

dict_keys(['2014-11', '2014-12', '2015-01', '2015-02', '2015-03'])

In [11]:
len(data.groupby(['month']).groups['2014-11'])

230

In [12]:
# Get the first entry for each month
data.groupby('month').first()

index                date  duration  item   network network_type
month                                                                    
2014-11      0 2014-10-15 06:58:00    34.429  data      data         data
2014-12    228 2014-11-13 06:58:00    34.429  data      data         data
2015-01    381 2014-12-13 06:58:00    34.429  data      data         data
2015-02    577 2015-01-13 06:58:00    34.429  data      data         data
2015-03    729 2015-02-12 20:15:00    69.000  call  landline     landline

In [13]:
# Get the sum of the durations per month
data.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [14]:
# Get the number of dates / entries in each month
data.groupby('month')['date'].count()

month
2014-11    230
2014-12    157
2015-01    205
2015-02    137
2015-03    101
Name: date, dtype: int64

In [15]:
# What is the sum of durations, for calls only, to each network
data[data['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

You can also group by more than one variable, allowing more complex queries.

In [16]:
# How many calls, sms, and data entries are in each month?
data.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [17]:
# How many calls, texts, and data are sent per month, split by network_type?
data.groupby(['month', 'network_type'])['date'].count()

month    network_type
2014-11  data             29
         landline          5
         mobile          189
         special           1
         voicemail         6
2014-12  data             30
         landline          7
         mobile          108
         voicemail         8
         world             4
2015-01  data             31
         landline         11
         mobile          160
         voicemail         3
2015-02  data             31
         landline          8
         mobile           90
         special           2
         voicemail         6
2015-03  data             29
         landline         11
         mobile           54
         voicemail         4
         world             3
Name: date, dtype: int64

##### Groupby output format – Series or DataFrame?

The output from a groupby and aggregation operation varies between Pandas Series and Pandas Dataframes. 
- As a rule of thumb, if you calculate more than one column of results, your result will be a Dataframe.
- For a single column of results, the agg function, by default, will produce a Series.

You can change this by selecting your operation column differently:

In [18]:
# produces Pandas Series
data.groupby('month')['duration'].sum() 

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [19]:
# Produces Pandas DataFrame
data.groupby('month')[['duration']].sum()

duration
month             
2014-11  26639.441
2014-12  14641.870
2015-01  18223.299
2015-02  15522.299
2015-03  22750.441

The groupby output will have an index or multi-index on rows corresponding to your chosen grouping variables. To avoid setting this index, pass `as_index=False` to the groupby operation.

In [20]:
data.groupby('month', as_index=False).agg({"duration": "sum"})

month   duration
0  2014-11  26639.441
1  2014-12  14641.870
2  2015-01  18223.299
3  2015-02  15522.299
4  2015-03  22750.441

### Aggregation: Applying a single function to columns in groups

Instructions for aggregation are provided in the form of a python dictionary or list. The dictionary keys are used to specify the columns upon which you’d like to perform operations, and the dictionary values to specify the function to run.

In [21]:
# Group the data frame by month and item and extract a number of stats from each group
data.groupby(
   ['month', 'item']
).agg(
    {
         'duration':sum,    # Sum duration per group
         'network_type': "count",  # get the count of networks
         'date': 'first'  # get the first date per group
    }
)

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

### Aggregating: Applying multiple functions to columns in groups

In [22]:
# Group the data frame by month and item and extract a number of stats from each group
data.groupby(
    ['month', 'item']
).agg(
    {
        # find the min, max, and sum of the duration column
        'duration': [min, max, sum],
         # find the number of network type entries
        'network_type': "count",
        # min, first, and number of unique dates per group
        'date': [min, 'first', 'nunique']
    }
)

duration                       network_type                date  \
                  min        max        sum        count                 min   
month   item                                                                   
2014-11 call    1.000   1940.000  25547.000          107 2014-10-15 06:58:00   
        data   34.429     34.429    998.441           29 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000           94 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000           79 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870           30 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000           48 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000           88 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299           31 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000           86 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000           67 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299           31 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000           39 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000           47 2015-02-12 20:15:00   
        data   34.429     34.429    998.441           29 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000           25 2015-02-19 18:46:00   

                                          
                           first nunique  
month   item                              
2014-11 call 2014-10-15 06:58:00     104  
        data 2014-10-15 06:58:00      29  
        sms  2014-10-16 22:18:00      79  
2014-12 call 2014-11-14 17:24:00      76  
        data 2014-11-13 06:58:00      30  
        sms  2014-11-14 17:28:00      41  
2015-01 call 2014-12-15 20:03:00      84  
        data 2014-12-13 06:58:00      31  
        sms  2014-12-15 19:56:00      58  
2015-02 call 2015-01-15 10:36:00      67  
        data 2015-01-13 06:58:00      31  
        sms  2015-01-15 12:23:00      27  
2015-03 call 2015-02-12 20:15:00      47  
        data 2015-02-13 06:58:00      29  
        sms  2015-02-19 18:46:00      17

### Renaming grouped statistics from groupby operations

When multiple statistics are calculated on columns, the resulting dataframe will have a multi-index set on the column axis.
This may be difficult to work with.

One option is to drop the top level (using `.droplevel`) of the newly created multi-index on columns using:

In [23]:
grouped = data.groupby('month').agg({'duration':[min, max]})
grouped.columns = grouped.columns.droplevel(level=0)
grouped.rename(columns={
    "min": "min_duration", "max": "max_duration"
})
grouped.head()

min      max
month                
2014-11  1.0   1940.0
2014-12  1.0   2120.0
2015-01  1.0   1859.0
2015-02  1.0   1863.0
2015-03  1.0  10528.0

However, this approach loses the original column names, leaving only the function names as column headers.

A neatful method is using the `ravel()` method on the grouped columns. `ravel()` turns a Pandas multi-index into a simpler array, which we can combine into sensible column names

In [24]:
grouped = data.groupby('month').agg({"duration": [min, max]}) 
# Using ravel, and a string join, we can create better names for the columns:
grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]

grouped.head()

duration_min  duration_max
month                              
2014-11           1.0        1940.0
2014-12           1.0        2120.0
2015-01           1.0        1859.0
2015-02           1.0        1863.0
2015-03           1.0       10528.0